In [1]:
from pydisagg.disaggregate import split_datapoint
from pydisagg.disaggregate import split_dataframe


import numpy as np
from pydisagg.models import RateMultiplicativeModel
from pydisagg.models import LMOModel
from pydisagg.models import LogOddsModel
import pandas as pd
from example_df_dataset import data_df,population_sizes,baseline_patterns,groups_to_split_into

In [2]:
groups_to_split_into

['age_group_0', 'age_group_1', 'age_group_2', 'age_group_3']

In [3]:
display(data_df,population_sizes,baseline_patterns)

,demographic_id,pattern_id,age_group_0,age_group_1,age_group_2,age_group_3,obs,obs_se
0,0,0,1,1,0,0,7.0,1.0
1,0,0,0,0,1,1,20.0,2.0
2,1,0,1,1,1,0,10.0,3.0
3,1,0,0,0,0,1,11.0,1.0
4,2,1,1,1,1,1,10.0,1.5
5,3,1,1,0,0,0,0.8,0.1
6,3,1,0,1,0,0,8.0,1.0
7,3,1,0,0,1,1,22.0,3.0


,age_group_0,age_group_1,age_group_2,age_group_3
demographic_id,,,,
0,10,10,20,20
1,10,20,20,13
2,20,20,10,10
3,5,40,30,20


,age_group_0,age_group_1,age_group_2,age_group_3
pattern_id,,,,
0,0.4,0.20,0.3,0.75
1,0.1,0.15,0.2,0.70


In [4]:
split_dataframe(
    groups_to_split_into,
    data_df,
    population_sizes,
    baseline_patterns
)

,demographic_id,age_group_0,age_group_1,age_group_2,age_group_3
0,0,4.587886,2.412114,0.000000,0.000000
1,0,0.000000,0.000000,5.485838,14.514162
2,1,2.976175,2.742213,4.281612,0.000000
3,1,0.000000,0.000000,0.000000,11.000000
4,2,1.195737,1.834592,1.251663,5.718007
5,3,0.800000,0.000000,0.000000,0.000000
6,3,0.000000,8.000000,0.000000,0.000000
7,3,0.000000,0.000000,7.121403,14.878597


In [5]:
df=split_dataframe(
    groups_to_split_into,
    data_df,
    population_sizes,
    baseline_patterns,
    use_se=True
)
df

demographic_id    estimate                                              se  \
                 age_group_0 age_group_1 age_group_2 age_group_3 age_group_0   
0              0    4.587886    2.412114    0.000000    0.000000    0.575665   
1              0    0.000000    0.000000    5.485838   14.514162    0.000000   
2              1    2.976175    2.742213    4.281612    0.000000    0.801781   
3              1    0.000000    0.000000    0.000000   11.000000    0.000000   
4              2    1.195737    1.834592    1.251663    5.718007    0.266241   
5              3    0.800000    0.000000    0.000000    0.000000    0.100000   
6              3    0.000000    8.000000    0.000000    0.000000    0.000000   
7              3    0.000000    0.000000    7.121403   14.878597    0.000000   

                                       
  age_group_1 age_group_2 age_group_3  
0    0.424335    0.000000    0.000000  
1    0.000000    1.000000    1.000000  
2    0.907569    1.290650    0.000000  
3    0.000000    0.000000    1.000000  
4    0.394610    0.259314    0.579834  
5    0.000000    0.000000    0.000000  
6    1.000000    0.000000    0.000000  
7    0.000000    1.763117    1.236883

In [6]:
df_rate=split_dataframe(
    groups_to_split_into,
    data_df,
    population_sizes,
    baseline_patterns,
    use_se=True,
    output_type='rate'
)
df_rate

demographic_id    estimate                                              se  \
                 age_group_0 age_group_1 age_group_2 age_group_3 age_group_0   
0              0    0.458789    0.241211    0.000000    0.000000    0.057566   
1              0    0.000000    0.000000    0.274292    0.725708    0.058568   
2              1    0.297618    0.137111    0.214081    0.000000    0.080178   
3              1    0.000000    0.000000    0.000000    0.846154    0.146250   
4              2    0.059787    0.091730    0.125166    0.571801    0.013312   
5              3    0.160000    0.000000    0.000000    0.000000    0.020000   
6              3    0.000000    0.200000    0.000000    0.000000    0.018360   
7              3    0.000000    0.000000    0.237380    0.743930    0.034659   

                                       
  age_group_1 age_group_2 age_group_3  
0    0.042434    0.052932    0.038152  
1    0.037179    0.050000    0.050000  
2    0.045378    0.064533    0.086557  
3    0.127347    0.145600    0.076923  
4    0.019730    0.025931    0.057983  
5    0.026535    0.031250    0.023810  
6    0.025000    0.030178    0.027862  
7    0.047946    0.058771    0.061844

## Set up a problem with mean type observation

In [27]:
pops_in_measurement = (data_df.set_index('demographic_id')[groups_to_split_into]*population_sizes*1.).sum(axis=1).reset_index(drop=True)
norm_data_df= data_df.copy()
norm_data_df['obs'] = data_df['obs']/pops_in_measurement
norm_data_df['obs_se'] = data_df['obs_se']/pops_in_measurement

In [30]:
df_rate_normalized=split_dataframe(
    groups_to_split_into,
    data_df,
    population_sizes,
    baseline_patterns,
    use_se=True,
    output_type='rate',
    normalize_pop_for_average_type_obs=True
)
df_rate_normalized

demographic_id    estimate                                              se  \
                 age_group_0 age_group_1 age_group_2 age_group_3 age_group_0   
0              0    0.458789    0.241211    0.000000    0.000000    0.057566   
1              0    0.000000    0.000000    0.274292    0.725708    0.058568   
2              1    0.297618    0.137111    0.214081    0.000000    0.080178   
3              1    0.000000    0.000000    0.000000    0.846154    0.146250   
4              2    0.059787    0.091730    0.125166    0.571801    0.013312   
5              3    0.160000    0.000000    0.000000    0.000000    0.020000   
6              3    0.000000    0.200000    0.000000    0.000000    0.018360   
7              3    0.000000    0.000000    0.237380    0.743930    0.034659   

                                       
  age_group_1 age_group_2 age_group_3  
0    0.042434    0.052932    0.038152  
1    0.037179    0.050000    0.050000  
2    0.045378    0.064533    0.086557  
3    0.127347    0.145600    0.076923  
4    0.019730    0.025931    0.057983  
5    0.026535    0.031250    0.023810  
6    0.025000    0.030178    0.027862  
7    0.047946    0.058771    0.061844